# Finding names in book reviews

In [1]:
#import necessary packages
import os
import pandas as pd
import re
from collections import Counter
import numpy as np
import pickle
from nltk.metrics import edit_distance
%pprint

Pretty printing has been turned OFF


In [2]:
from nltk.corpus import stopwords
stopword_list = stopwords.words('english')

In [3]:
# loading in files
directory = "../../aps_reviews_50/aps_reviews/"
filenames = os.listdir(directory)
txts = []
for file in filenames:
    with open(directory + file) as f:
        txts.append(f.read())

In [4]:
df_all = pickle.load(open('author_title_df.pkl', 'rb'))

In [5]:
authors_by_last_name = pickle.load(open('author_last_name_dict.pkl', 'rb'))

### Step 1: Find names using titles and capitalization

In [6]:
titles = """Doctor,Dr,Mr,Mrs,Miss,Msgr,Monsignor,Rev,Reverend,Hon,Honorable,Honourable,Prof,Professor,Madame,Madam,Lady,Lord,Sir,Dame,Master,Mistress,Chancellor,Principal,President,Pres,Warden,Dean,Regent,Rector,Provost,Director
"""

In [7]:
titles = titles.rstrip().split(',')

In [8]:
#title_list = r"\.? .+? |".join(titles)

In [9]:
title_list = '.?\s|'.join(titles)

In [10]:
# test text
mytext = """
    Missouri: A Bone of Contention '                 WE  aware that most of the States of the Union had their nicknames, more or less complimentary, but to name Missouri ' a bone of contention ' is a stroke of wit. It does, however, rightly describe the Missouri of the past, and vividly writes in a phrase her political history. Until the triumph of the Union armies and the close of the Civil War, Missouri was in the jaws of the watch-dogs of slavery and freedom. In war or in peace, the subiect of legislative com-                 promise or of military struggle, Missouri was an uncertain factor. Now, after -five years of national peace, her history may be calmly and impartially written. Indeed, the task has been done, and well done, and the , Lucien Carr of Harvard, may be congratulated upon his work,  is strong, unimpassioned, scholarly, and as impressed with the firm touch which comes of local knowledge as are the imprinted rocks in the cabinets at Cambridge. Long familiarity with the wealth of archaeology in the Peabody Museuml seems to have given him the power of comparison and generalization in the evolution of a commonwealth, while  acquaintance with living men enables him to blend the results of the study and the field in pleasing literary form. Five of his seventeen chapters give a luminous picture of the early French and Spanish discoveries and domination. Then follow three chapters treating of the                 Missouri. By Ldclen Carr. $ti... (American Commonwealths.) Boston:                 Itoughto., Sftfltn & Co.                 l , the compromise, and the  into the U nion of this State named after the great river which flows through it. In his treatment of the period from 1844 to i861, as well as that of war time, some readers may charge Mr. Carr with unduly favoring the Southern and even Confederate view; but to people living this side of the now-vanished Mason and Dixon's line, this is doubtless a benefit; for only when Northern people are able to ' put themselves in the place' of Southerners and see with Southern eyes, can they be sure that they have achieved that impartiality which is essential to the writing of final history. He shows that the Missourians were neither secessionists nor slavery propagandists. He both criticises and justifies the action of the second convention which, in the uncertain hours when other States  seceding and Missouri's Governor had been driven into exile, org  a provisional government, and  saved Missouri Irons ' the pit of political degradation into which the States in rebellion were sunk during the period of reconstruction.' Mr. Carr practically and almost abruptly ends his history at the close of the War, believing that the career of Missouri as a bone of contention ended with the abolition of slavery. The fifty years' struggle was over, the State recovered rapidly froni the wounds of the Civil War, wealth increased wonderfully, and the Negro was liberally dealt with in most if not all points relating to citizenship. Taken as a whole, this book, with its sustained interest, high average literary merit, and thorough treatment of the voluminous facts, fully justifies its place in the series of ' histories of such States as have exercised a positive influence in the shaping of the national Government, or have had a striking political - . . history.' Like the others, it has a good map and index."""  

In [11]:
from symspellpy.symspellpy import SymSpell, Verbosity 

# maximum edit distance per dictionary precalculation
max_edit_distance_dictionary = 2
prefix_length = 7
# create object
first_name_symspell = SymSpell(max_edit_distance_dictionary, prefix_length)
# load dictionary
dictionary_path = "name_freq_dict.txt"
term_index = 0 
count_index = 1
first_name_symspell.load_dictionary(dictionary_path, term_index, count_index, separator=",")

True

In [12]:
authors_last_name_list = ','.join(list(authors_by_last_name.keys()))

In [13]:
f = open("authors_last_name_list.txt", "w")
f.write(authors_last_name_list)
f.close()

In [14]:
# maximum edit distance per dictionary precalculation
max_edit_distance_dictionary = 2
prefix_length = 7
# create object
author_surname_symspell = SymSpell()
# load dictionary
term_index = 0 
count_index = 1
author_surname_symspell.create_dictionary('authors_last_name_list.txt')

True

In [15]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [16]:
def cleanTextForNameSearch(txt):
    """
    Removes all non-newline whitespace and adds spaces around commas, semicolons, and colons.
    """
    #delete extra whitespace
    txt = re.sub(' +',' ',txt)
    
    #delete characters that should never be in this dataset (i think)
    txt = re.sub("\\'\(\)\*/<=>@\[\]^_`\|~","",txt)
    
    #adding space around certain punctuation
    txt = re.sub(',',' , ',txt)
    txt = re.sub(';',' ; ',txt)
    txt = re.sub(':',' : ',txt)
    txt = re.sub('"',' " ',txt)
    txt = re.sub("'"," ' " ,txt)
    return txt

notes:
+ reduce names down to 1880-1900
+ rule for 's

In [17]:
def removePeriodsNotFollowingTitleOrInitial(name):
    name_parts = name.split()
    cleaned_name = []
    for part in name_parts:
        if (len(part)>2) and (part.endswith('.')) and (part[:-1] not in titles):
            cleaned_name.append(part[:-1])
        else:
            cleaned_name.append(part)
    return ' '.join(cleaned_name)

In [18]:
def cleanIndices(index_tuple):
    x = index_tuple[0]
    y = index_tuple[1]
    if y[-1] in '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~':
        y = y[:-1]
    return (x,y)

In [45]:
def cleanName(name):
    return ' '.join([word for word in name.split() if (word[0].isalpha())])

In [46]:
def getNamesFollowingTitles(txt):
    """
    Returns names following titles - specifically capitalized titles followed by capitalized names.
    Names can be any number of words in length, and can include punctuation.
    """
    names = []
    
    # iter only works once before emptying
    iterx = re.finditer(title_list, txt)
    indices = [(m.start(), m.group()) for m in iterx]
    indices = [cleanIndices(m) for m in indices]
    
    for i, index in enumerate(indices):
        if (i<len(indices)-1):
            try:
                match = re.match(indices[i][1] + '\w{2,}[.,;:!\?\'\"]', txt[indices[i][0]:indices[i+1][0]])
                names.append(match.group()[:-1])
            except:
                try:
                    match = re.match(indices[i][1] + '.*? [a-z]|\Z', 
                                     txt[indices[i][0]:indices[i+1][0]])
                    names.append(match.group()[:-2])
                except:
                    pass
        else:
            try:
                match = re.match(indices[i][1] + '\w{2,}[.,;:!\?\'\"]', txt[indices[i][0]:])
                names.append(match.group()[:-1])
            except:
                try:
                    match = re.match(indices[i][1] + '.*? [a-z]|\Z', 
                                     txt[indices[i][0]:])
                    names.append(match.group()[:-2])
                except:
                    pass
    
    names = [word.replace("'s", "") for word in names]
    names = [removePeriodsNotFollowingTitleOrInitial(word) for word in names]
    names = [cleanName(word) for word in names]
    
    return list(set(names))

In [21]:
def removePunct(word):
    return ''.join([x for x in word if (x.isalnum()) or (x == '-') or (x=='–')])

In [22]:
def getCapitalizedWords(txt):
    """
    Returns strings of capitalized words up 3 words long. 
    Removes words/phrases containing stopwords and words found later in the text lowercased.
    """
    #listen idk why it won't just let me put in an optional repeat either
    all_words = []
    
    #three words
    all_words.extend([match for match in re.findall('[A-Z]\S* [A-Z]\S* [A-Z]\S+', txt) if 
                      all([(removePunct(word).lower() not in stopword_list) for word in match.split()]) 
                      and all([(removePunct(word).lower() not in txt) for word in match.split()])])
    
    #two words
    two_words = [match for match in re.findall('[A-Z]\S* [A-Z]\S+', txt) if 
                      all([(removePunct(word).lower() not in stopword_list) for word in match.split()]) 
                      and all([(removePunct(word).lower() not in txt) for word in match.split()])
                      and all([match not in x for x in all_words])]
    
    all_words.extend(two_words)
    
    #one word
    one_words = [match for match in re.findall('[A-Z]\S+', txt) if 
                      (removePunct(match).lower() not in stopword_list) 
                      and (removePunct(match).lower() not in txt)
                      and all([match not in x for x in all_words])]
    
    all_words.extend(one_words)
    
    return [word for word in [' '.join([removePunct(y) for y in x.split() if removePunct(y) not in titles]) 
                      for x in all_words] if (len(word)>1)]

In [23]:
from nltk.metrics import edit_distance

In [24]:
edit_distance('voynich','voynicch')

1

In [47]:
exlist = getNamesFollowingTitles(cleanTextForNameSearch(txts[0]))

In [48]:
exlist

['Professor Thorold Rogers', 'Mrs. Browning', 'Mr. Julian Hawthorne', 'Mr. Vedder', 'Mr. Swinburne', 'President Andrew D. White', 'Dr. Geor6 Ebers', 'Mr. Julius Chambers', 'Professor E. Johnsou', 'Dr. William M. Taylor', 'Dr. Joseph Parktercontinueshiseipositionuin', 'Dr. i. B. Adams', 'Mr. Brander Matthews', 'Mr. Cushing', 'Dr. Herbert B. Adam', 'Mr. Charles Dudley Warner', 'Dr. Parker']

In [60]:
for x in exlist:
    print(x, len(x.split()))

Professor Thorold Rogers 3
Mrs. Browning 2
Mr. Julian Hawthorne 3
Mr. Vedder 2
Mr. Swinburne 2
President Andrew D. White 4
Dr. Geor6 Ebers 3
Mr. Julius Chambers 3
Professor E. Johnsou 3
Dr. William M. Taylor 4
Dr. Joseph Parktercontinueshiseipositionuin 3
Dr. i. B. Adams 4
Mr. Brander Matthews 3
Mr. Cushing 2
Dr. Herbert B. Adam 4
Mr. Charles Dudley Warner 4
Dr. Parker 2


In [82]:
longname = 'Mr. Charles E. B. Dudley Warner'.split()

In [84]:
[(len(x)<3, x) for x in longname[1:-1]]

[(False, 'Charles'), (True, 'E.'), (True, 'B.'), (False, 'Dudley')]

In [78]:
len('E.') < 3

True

In [62]:
' '.join(['egg','egg'])

'egg egg'

In [89]:
egg = ''

In [90]:
if egg:
    print(x)

In [109]:
class AuthName(str):
    """
    I have never made a class before. I hope I'm doing this right.
    """
    name_type = 'author'
        
    def __assign(self):
        
        # just title & last name
        if self.name_part_count < 3:
            pass
        
        # title, first name/initial, last name
        elif self.name_part_count == 3:
            if (len(self.name_parts[1])) < 3:
                self.first_initial = self.name_parts[1]
            else:
                self.first_name = self.name_parts[1]
        
        elif self.name_part_count == 4:
            if (len(self.name_parts[1])) < 3 and (len(self.name_parts[2])) < 3:
                self.initials = ' '.join([self.name_parts[1], self.name_parts[1]])
            else:
                if (len(self.name_parts[1])) < 3:
                    self.first_initial = self.name_parts[1]
                    self.middle_name = self.name_parts[2]
                else:
                    self.first_name = self.name_parts[1]
                    self.middle_initial = self.name_parts[2]
                
        elif self.name_part_count > 4:
            
            # all initials
            if all([len(x)<3 for x in self.name_parts[1:-1]]):
                initial_list = []
                for x in self.name_parts[1:-1]:
                    initial_list.append(x)
                self.initials = ' '.join(initial_list)
                
            # all names
            elif all([len(x)>2 for x in self.name_parts[1:-1]]):
                self.first_name = self.name_parts[1]
                middle_name_list = []
                for x in self.name_parts[1:-1]:
                    middle_name_list.append(x)
                self.middle_name = ' '.join(middle_name_list)
                
            # first initial, middle names
            elif (len(self.name_parts[1])<3 and all([len(x)>2 for x in self.name_parts[2:-1]])): 
                self.first_initial = self.name_parts[1]
                middle_name_list = []
                for x in self.name_parts[2:-1]:
                    middle_name_list.append(x)
                self.middle_name = ' '.join(middle_name_list)
                
            # first name, middle initials
            elif (len(self.name_parts[1])>2 and all([len(x)<3 for x in self.name_parts[2:-1]])): 
                self.first_name = self.name_parts[1]
                middle_initial_list = []
                for x in self.name_parts[2:-1]:
                    middle_initial_list.append(x)
                self.middle_initial = ' '.join(middle_initial_list)
                
    def __generate(self):
        if self.first_name:
            self.first_initial = self.first_name[0]
        if self.middle_name:
            self.middle_initial = [x[0] for x in self.middle_name[0].split()]
        if self.first_name and self.middle_name:
            self.initials = [x[0] for x in self.name_parts[1:-1].split()]
                
    def __fixInitials(initials):
        i_list = initials.split()
        i_list = [cleanPunct(x) for x in i_list]
        return ';'.join(i_list)
                
    def __reformat(self):
        if self.initials:
            self.initials = __fixInitials(self.initials)
        if self.middle_initial:
            self.middle_initial = __fixInitials(self.middle_initial)
        if self.first_initial:
            self.first_initial = __fixInitials(self.first_initial)
            
    def __init__(self, name):
        self.full_name = name
        self.name_parts = [x.lower() for x in self.full_name.split()]
        self.title = self.name_parts[0]
        self.last_name = self.name_parts[0]
        self.name_part_count = len(self.name_parts)
        
        self.first_name = ''
        self.first_initial = ''
        self.middle_name = ''
        self.middle_initial = ''
        self.initials = ''
        
        self.__assign()
        self.__generate()
        self.__reformat()
            
                    
    #__assign = assign
    #__generate = generate
    #__reformat = reformat
        

In [106]:
name = AuthName('Mr. Charles E. B. Dudley Warner')

In [107]:
name.first_name

''

In [52]:
author_surname_symspell.lookup('adams', Verbosity.CLOSEST)[0]._term

'adams'

In [54]:
def getSetOfNames(txt):
    """
    Returns sets of names matched by similarity.
    """
    all_name_sets = []
    
    known_names = getNamesFollowingTitles(txt)
    potential_names = getCapitalizedWords(txt)
    
    for known in known_names:
        name_parts = [removePunct(x).lower() for x in known.split() if removePunct(x) not in titles]
        name_set = []
        name_set.append(known)

        
        try:
            author_surname_symspell._words[name_parts[-1].lower()]
            for n in potential_names:
                lookup = author_surname_symspell.lookup(n.lower(), Verbosity.CLOSEST)
                if (lookup):
                    for i, x in enumerate(lookup):
                        if (lookup[i]._term in name_parts):
                            name_set.append(n)
        
        except:
            for n in potential_names:
                for p in name_parts:
                    if (len(n)>4) and (len(p)>4):
                        if (edit_distance(n,p)<3):
                            name_set.append(n)
        
        all_name_sets.append(name_set)
        
    return all_name_sets

In [59]:
getSetOfNames(txts_cleaned[0])

[['Professor Thorold Rogers'], ['Mrs. Browning', 'Browning'], ['Mr. Julian Hawthorne'], ['Mr. Vedder', 'Vedder'], ['Mr. Swinburne', 'Swinburne'], ['President Andrew D. White', 'White'], ['Dr. Geor6 Ebers', 'Ebers'], ['Mr. Julius Chambers'], ['Professor E. Johnsou', 'Johnsou'], ['Dr. William M. Taylor', 'Taylor'], ['Dr. Joseph Parktercontinueshiseipositionuin', 'Joseph', 'Parktercontinueshiseipositionuin'], ['Dr. i. B. Adams', 'Adams'], ['Mr. Brander Matthews'], ['Mr. Cushing', 'Cushing'], ['Dr. Herbert B. Adam', 'Herbert', 'Adam'], ['Mr. Charles Dudley Warner'], ['Dr. Parker', 'Parker']]

In [56]:
txts_cleaned = [cleanTextForNameSearch(x) for x in txts]

if first part of shorter name is entirely contained in longer name
fuzzy match to edit distance of two 
look at part of string length of shorter name - if those fuzzy match
cut it off and group, check to see if second part is dictionary name

match known publishers
remove publishers
regex with name & name & name

In [57]:
for num, txt in enumerate(txts_cleaned):
    try:
        print('Text #' + str(num))
        for x in getSetOfNames(txt):
            print(x)
        print()
    except:
        print('Error')
        print()

Text #0
['Professor Thorold Rogers']
['Mrs. Browning', 'Browning']
['Mr. Julian Hawthorne']
['Mr. Vedder', 'Vedder']
['Mr. Swinburne', 'Swinburne']
['President Andrew D. White', 'White']
['Dr. Geor6 Ebers', 'Ebers']
['Mr. Julius Chambers']
['Professor E. Johnsou', 'Johnsou']
['Dr. William M. Taylor', 'Taylor']
['Dr. Joseph Parktercontinueshiseipositionuin', 'Joseph', 'Parktercontinueshiseipositionuin']
['Dr. i. B. Adams', 'Adams']
['Mr. Brander Matthews']
['Mr. Cushing', 'Cushing']
['Dr. Herbert B. Adam', 'Herbert', 'Adam']
['Mr. Charles Dudley Warner']
['Dr. Parker', 'Parker']

Text #1
['Mr. E. L. Voynich', 'Voynich', 'VWynich', 'Voyniich']

Text #2
['Miss Peard', 'Peard', 'Peard']
['Sir Henry Lancaster', 'Lancaster', 'Lancaster', 'Lancaster']
['Miss HowAiD Is']
['Lord Mttdhursm- Coming']
['Miss s', 'Shp', 'S8', 'G0']
['Mrs.']
['Mrs. A. L. Wister', 'Wister', 'Ap-', 'L8', 'G0', 'G0']

Text #3
['Dr. Atwood', 'Atwood']

Text #4

Text #5
['Lord']

Text #6
['Mr. Thomas A. Edison', 'Thomas'